<a href="https://colab.research.google.com/github/bruno-teider/BigData/blob/main/Airports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession
import re

spark = SparkSession.builder.appName("airport").master("local[*]").getOrCreate()

sc = spark.sparkContext

In [ ]:
rdd = sc.textFile("Airports.csv")

# First Task: All brazilian Airports with latitude > 25
# oder dec and save in file "Brazil_airports_lat25.txt"
rdd_brazil = rdd.filter(lambda x: x.split(",")[4] == "BRAZIL")

rdd_lat = rdd_brazil.filter(lambda x: int(x.split(",")[5]) > 25)

rdd_lat.count()

rdd_lower = rdd_lat.map(lambda x: x.lower())

rdd_final = rdd_lower.sortBy(lambda x: int(x.split(",")[5]), ascending = False)

rdd_final.coalesce(1).saveAsTextFile("Brazil_airports_lat25.txt")

In [41]:
# Second Task: Calculate medium lat of brazilian airports and standard desviation
rdd_brazil = rdd.filter(lambda x: x.split(",")[4] == "BRAZIL")
rdd_lat = rdd_brazil.map(lambda x: int(x.split(",")[5]))

sum = rdd_lat.reduce(lambda x,y : x + y)

n = rdd_lat.map(lambda x: 1).reduce(lambda x,y : x + y)

medium = sum/n

print(f'Sum of lat: {sum} / Number of Airports: {n} = {medium:.2f}')

# Standard Desviation
variation = rdd_lat.map(lambda x: (x - medium)**2).reduce(lambda x,y : x + y)

std = (variation/n)**0.5
print(f'Standard Variation: {std:.2f}')

Sum of lat: 1962 / Number of Airports: 142 = 13.82
Standard Variation: 10.06


In [69]:
# Third Task: Biggest and lowest alt
rdd = rdd.filter(lambda x: x.split(",")[0] != "icao_code")
rdd_alt = rdd.map(lambda x: int(x.split(",")[13]))

max = rdd_alt.reduce(lambda x,y : x if x > y else y)
min = rdd_alt.reduce(lambda x,y : x if x < y else y)

print(f'Max: {max}')
print(f'Min: {min}')

max_airport = rdd.filter(lambda x: int(x.split(",")[13]) == max)
min_airport = rdd.filter(lambda x: int(x.split(",")[13]) == min)

print("Max Airport: ")
max_airport.take(1)

Max: 4060
Min: -385
Max Airport: 


['SLCN,N/A,CHARANA,CHARANA,BOLIVIA,17,35,38,S,69,26,3,W,4060,-17.594,-69.434,7551']

In [70]:
print("Min Airport: ")
min_airport.take(1)

Min Airport: 


['LLMZ,MTZ,BAR YEHUDA,METZADA,ISRAEL,31,19,41,N,35,23,18,E,-385,31.328,35.388,4813']